- link to data source: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
- link to  data dictionary: https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf

In [29]:
import pandas as pd

In [30]:

df = pd.read_parquet('../data/yellow_tripdata_2016-03.parquet')



In [12]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2016-03-01 00:53:35,2016-03-01 00:57:11,1,0.0,1,N,226,226,3,4.0,0.5,0.5,0.0,0.0,0.3,5.3,None,None
1,1,2016-03-01 00:15:13,2016-03-01 00:15:26,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,None,None
2,1,2016-03-01 00:15:46,2016-03-01 00:16:00,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,None,None
3,1,2016-03-01 00:16:17,2016-03-01 00:16:23,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,None,None
4,1,2016-03-01 00:16:45,2016-03-01 00:18:14,1,0.0,1,N,145,145,2,3.0,0.5,0.5,0.0,0.0,0.3,4.3,None,None


In [25]:
if 'tpep_pickup_datetime' in df.columns: df['tpep_pickup_datetime'] = pd.to_datetime(df.tpep_pickup_datetime)


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12203824 entries, 0 to 12203823
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        int64         
 4   trip_distance          float64       
 5   RatecodeID             int64         
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   object        
 18  airport_fee         

In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [5]:
if 'tpep_pickup_datetime' in df.columns: df['tpep_pickup_datetime'] = pd.to_datetime(df.tpep_pickup_datetime)
if 'tpep_dropoff_datetime' in df.columns: df['tpep_dropoff_datetime'] = pd.to_datetime(df.tpep_dropoff_datetime)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12203824 entries, 0 to 12203823
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        int64         
 4   trip_distance          float64       
 5   RatecodeID             int64         
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   object        
 18  airport_fee         

In [7]:
#check duplicates: no duplicates row in raw data
df[df.duplicated()].shape

(0, 19)

# Create Data Model Ver. 1

In [26]:
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True)

datetime_dim = datetime_dim.assign(
    pickup_year = datetime_dim.tpep_pickup_datetime.dt.year,\
    pickup_month = datetime_dim.tpep_pickup_datetime.dt.month,\
    pickup_day = datetime_dim.tpep_pickup_datetime.dt.day,\
    pickup_weekday = datetime_dim.tpep_pickup_datetime.dt.weekday,\
    dropoff_year = datetime_dim.tpep_dropoff_datetime.dt.year,\
    dropoff_month = datetime_dim.tpep_dropoff_datetime.dt.month,\
    dropoff_day = datetime_dim.tpep_dropoff_datetime.dt.day,\
    dropoff_weekday = datetime_dim.tpep_dropoff_datetime.dt.weekday,\
    datetime_id = datetime_dim.index
)
 
datetime_dim = datetime_dim[[ 'datetime_id', 'tpep_pickup_datetime', 'pickup_year','pickup_month','pickup_day','pickup_weekday',
                             'tpep_dropoff_datetime', 'dropoff_year', 'dropoff_month', 'dropoff_day', 'dropoff_weekday']]
  
datetime_dim.head()    

,datetime_id,tpep_pickup_datetime,pickup_year,pickup_month,pickup_day,pickup_weekday,tpep_dropoff_datetime,dropoff_year,dropoff_month,dropoff_day,dropoff_weekday
0,0,2016-03-01 00:53:35,2016,3,1,1,2016-03-01 00:57:11,2016,3,1,1
1,1,2016-03-01 00:15:13,2016,3,1,1,2016-03-01 00:15:26,2016,3,1,1
2,2,2016-03-01 00:15:46,2016,3,1,1,2016-03-01 00:16:00,2016,3,1,1
3,3,2016-03-01 00:16:17,2016,3,1,1,2016-03-01 00:16:23,2016,3,1,1
4,4,2016-03-01 00:16:45,2016,3,1,1,2016-03-01 00:18:14,2016,3,1,1


In [28]:
datetime_dim.tpep_pickup_datetime.astype('string')
datetime_dim.tpep_dropoff_datetime.astype('string')

0           2016-03-01 00:57:11
1           2016-03-01 00:15:26
2           2016-03-01 00:16:00
3           2016-03-01 00:16:23
4           2016-03-01 00:18:14
                   ...         
12176714    2016-03-31 23:22:56
12176715    2016-03-31 23:28:30
12176716    2016-04-01 00:07:05
12176717    2016-03-31 23:38:22
12176718    2016-03-31 23:56:26
Name: tpep_dropoff_datetime, Length: 12176719, dtype: string

In [9]:
rate_code_dim = df[['RatecodeID']].drop_duplicates().reset_index(drop=True)

rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = rate_code_dim.assign(
    rate_code_id = rate_code_dim.index,\
    rate_code_name = rate_code_dim['RatecodeID'].map(rate_code_type)
)

rate_code_dim = rate_code_dim[['rate_code_id', 'RatecodeID', 'rate_code_name']]

rate_code_dim

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,3,Newark
2,2,2,JFK
3,3,5,Negotiated fare
4,4,4,Nassau or Westchester
5,5,6,Group ride
6,6,99,NaN


In [10]:
payment_type_dim = df[['payment_type']].drop_duplicates().reset_index(drop=True)

payment_types = {
    1 : "Credit Card",
    2 : 'Cash',
    3: 'No charge',
    4: 'Dispute',
    5 : 'Unknown',
    6 : 'Voided trip'
}

payment_type_dim = payment_type_dim.assign(
    payment_type_id = payment_type_dim.index,\
    payment_type_name = payment_type_dim.payment_type.map(payment_types)
    
)

payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]

payment_type_dim

,payment_type_id,payment_type,payment_type_name
0,0,3,No charge
1,1,2,Cash
2,2,1,Credit Card
3,3,4,Dispute


In [11]:
df_taxi_zones = pd.read_csv('../data/taxi_zones.csv', usecols=['zone', 'LocationID','borough'])

df_taxi_zones_lookup = df_taxi_zones.drop_duplicates()
df_taxi_zones_lookup = df_taxi_zones_lookup[['LocationID', 'borough','zone']]

dropoff_location_dim = df[['DOLocationID']].drop_duplicates().reset_index(drop=True)

dropoff_location_dim = dropoff_location_dim.assign(
    dropoff_location_id = dropoff_location_dim.index
)

dropoff_location_dim = dropoff_location_dim.merge(df_taxi_zones_lookup, how='left', left_on= 'DOLocationID', right_on='LocationID')

dropoff_location_dim = dropoff_location_dim.rename(columns={'zone' : 'dropoff_location_zone',
                                     'borough' : 'dropoff_location_borough'})

dropoff_location_dim = dropoff_location_dim[['dropoff_location_id', 'DOLocationID', 'dropoff_location_zone','dropoff_location_borough']]

dropoff_location_dim

,dropoff_location_id,DOLocationID,dropoff_location_zone,dropoff_location_borough
0,0,226,Sunnyside,Queens
1,1,145,Long Island City/Hunters Point,Queens
2,2,264,NaN,NaN
3,3,151,Manhattan Valley,Manhattan
4,4,74,East Harlem North,Manhattan
...,...,...,...,...
257,257,5,Arden Heights,Staten Island
258,258,2,Jamaica Bay,Queens
259,259,105,NaN,NaN
260,260,99,Freshkills Park,Staten Island


In [12]:
df_taxi_zones = pd.read_csv('../data/taxi_zones.csv', usecols=['zone', 'LocationID','borough'])

df_taxi_zones_lookup = df_taxi_zones.drop_duplicates()
df_taxi_zones_lookup = df_taxi_zones_lookup[['LocationID', 'borough','zone']]

pickup_location_dim = df[['PULocationID']].drop_duplicates().reset_index(drop=True)

pickup_location_dim = pickup_location_dim.assign(
    pickup_location_id = pickup_location_dim.index
)

pickup_location_dim = pickup_location_dim.merge(df_taxi_zones_lookup, how='left', left_on= 'PULocationID', right_on='LocationID')

pickup_location_dim = pickup_location_dim.rename(columns={'zone' : 'pickup_location_zone',
                                     'borough' : 'pickup_location_borough'})

pickup_location_dim = pickup_location_dim[['pickup_location_id', 'PULocationID', 'pickup_location_zone','pickup_location_borough']]

pickup_location_dim

,pickup_location_id,PULocationID,pickup_location_zone,pickup_location_borough
0,0,226,Sunnyside,Queens
1,1,145,Long Island City/Hunters Point,Queens
2,2,264,NaN,NaN
3,3,43,Central Park,Manhattan
4,4,237,Upper East Side South,Manhattan
...,...,...,...,...
257,257,59,Crotona Park,Bronx
258,258,187,Port Richmond,Staten Island
259,259,103,Governor's Island/Ellis Island/Liberty Island,Manhattan
260,260,204,Rossville/Woodrow,Staten Island


In [13]:
trip_distance_dim = df[['trip_distance']].drop_duplicates().reset_index(drop=True)

trip_distance_dim = trip_distance_dim.assign(
    trip_distance_id = trip_distance_dim.index
)

trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_distance']]
trip_distance_dim

,trip_distance_id,trip_distance
0,0,0.00
1,1,0.90
2,2,1.50
3,3,2.40
4,4,2.00
...,...,...
4493,4493,71.85
4494,4494,54.88
4495,4495,59.70
4496,4496,50.67


In [14]:
passenger_count_dim =  df[['passenger_count']].drop_duplicates().reset_index(drop=True)

passenger_count_dim = passenger_count_dim.assign(
    passenger_count_id = passenger_count_dim.index
)

passenger_count_dim = passenger_count_dim[['passenger_count_id', 'passenger_count']]

passenger_count_dim

,passenger_count_id,passenger_count
0,0,1
1,1,2
2,2,3
3,3,5
4,4,4
5,5,6
6,6,0
7,7,8
8,8,7
9,9,9


In [30]:
vendor_dim = df[['VendorID']].drop_duplicates().reset_index(drop=True)

vendor_names = {
    1 : "Creative Mobile Technologies",
    2 : 'VeriFone Inc'
}

vendor_dim = vendor_dim.assign(
    vendor_id = vendor_dim['VendorID'].index,\
    vendor_name = vendor_dim.VendorID.map(vendor_names)
)

vendor_dim = vendor_dim[['vendor_id','VendorID', 'vendor_name']]

vendor_dim

,vendor_id,VendorID,vendor_name
0,0,1,Creative Mobile Technologies
1,1,2,VeriFone Inc


In [31]:
fct_table = df\
    .merge(vendor_dim[['vendor_id', 'VendorID']], how='left', on='VendorID')\
    .merge(datetime_dim[['datetime_id','tpep_pickup_datetime','tpep_dropoff_datetime']], how='left', on=['tpep_pickup_datetime','tpep_dropoff_datetime'])\
    .merge(rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']], how='left', on=['RatecodeID'])\
    .merge(payment_type_dim[['payment_type_id','payment_type','payment_type_name']], how='left', on=['payment_type'])\
    .merge(dropoff_location_dim[['dropoff_location_id','DOLocationID','dropoff_location_zone', 'dropoff_location_borough']], how='left', on=['DOLocationID'])\
    .merge(pickup_location_dim[['pickup_location_id','PULocationID','pickup_location_zone', 'pickup_location_borough']], how='left', on=['PULocationID'])\
    .merge(trip_distance_dim[['trip_distance_id','trip_distance']], how='left', on=['trip_distance'])\
    .merge(passenger_count_dim[['passenger_count_id','passenger_count']], how='left', on=['passenger_count'])\
    .assign(event_id = lambda x: x.index)
        
columns_in_fct_table = ['event_id','vendor_id', 'datetime_id','passenger_count_id','trip_distance_id','pickup_location_id',
                        'dropoff_location_id','rate_code_id','payment_type_id','store_and_fwd_flag', 'fare_amount', 'extra',
                        'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge','congestion_surcharge',
                        'airport_fee','total_amount']
    
fct_table = fct_table.loc[:, columns_in_fct_table]

fct_table

,event_id,vendor_id,datetime_id,passenger_count_id,trip_distance_id,pickup_location_id,dropoff_location_id,rate_code_id,payment_type_id,store_and_fwd_flag,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,airport_fee,total_amount
0,0,0,0,0,0,0,0,0,0,N,4.0,0.5,0.5,0.00,0.0,0.3,None,None,5.30
1,1,0,1,0,0,1,1,0,1,N,2.5,0.5,0.5,0.00,0.0,0.3,None,None,3.80
2,2,0,2,0,0,1,1,0,1,N,2.5,0.5,0.5,0.00,0.0,0.3,None,None,3.80
3,3,0,3,0,0,1,1,0,1,N,2.5,0.5,0.5,0.00,0.0,0.3,None,None,3.80
4,4,0,4,0,0,1,1,0,1,N,3.0,0.5,0.5,0.00,0.0,0.3,None,None,4.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12203819,12203819,1,12176714,0,33,29,23,0,2,N,15.0,0.5,0.5,1.63,0.0,0.3,None,None,17.93
12203820,12203820,1,12176715,0,250,41,24,0,1,N,6.0,0.5,0.5,0.00,0.0,0.3,None,None,7.30
12203821,12203821,1,12176716,0,751,21,38,0,2,N,21.5,0.5,0.5,4.56,0.0,0.3,None,None,27.36
12203822,12203822,1,12176717,0,705,43,52,0,2,N,12.0,0.5,0.5,1.00,0.0,0.3,None,None,14.30


In [34]:
#check any missing rows in fact table compared to raw data
print(f'Total row in raw data table: {df.shape[0]}')
print(f'Total row in fact table: {fct_table.shape[0]}')
print(f'Difference of rows between the two tables: {df.shape[0] - fct_table.shape[0]}\n')

display(fct_table.info())

Total row in raw data table: 12203824
Total row in fact table: 12203824
Difference of rows between the two tables: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12203824 entries, 0 to 12203823
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   event_id               int64  
 1   vendor_id              int64  
 2   datetime_id            int64  
 3   passenger_count_id     int64  
 4   trip_distance_id       int64  
 5   pickup_location_id     int64  
 6   dropoff_location_id    int64  
 7   rate_code_id           int64  
 8   payment_type_id        int64  
 9   store_and_fwd_flag     object 
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  congestion_surcharge   object 
 17  airport_fee            object 
 18  total_amount           float64
dtypes: flo

None

In [16]:
df.columns

[['store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge','congestion_surcharge' , 'airport_fee','total_amount']]

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

# Create Data Model Ver. 2

In [35]:
dim_datetime = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True)

dim_datetime = dim_datetime.assign(
    pickup_year = dim_datetime.tpep_pickup_datetime.dt.year,\
    pickup_month = dim_datetime.tpep_pickup_datetime.dt.month,\
    pickup_day = dim_datetime.tpep_pickup_datetime.dt.day,\
    pickup_weekday = dim_datetime.tpep_pickup_datetime.dt.weekday,\
    dropoff_year = dim_datetime.tpep_dropoff_datetime.dt.year,\
    dropoff_month = dim_datetime.tpep_dropoff_datetime.dt.month,\
    dropoff_day = dim_datetime.tpep_dropoff_datetime.dt.day,\
    dropoff_weekday = dim_datetime.tpep_dropoff_datetime.dt.weekday,\
    datetime_id = dim_datetime.index
)
 
dim_datetime = dim_datetime[[ 'datetime_id', 'tpep_pickup_datetime', 'pickup_year','pickup_month','pickup_day','pickup_weekday',
                             'tpep_dropoff_datetime', 'dropoff_year', 'dropoff_month', 'dropoff_day', 'dropoff_weekday']]
  
dim_datetime.head()    

,datetime_id,tpep_pickup_datetime,pickup_year,pickup_month,pickup_day,pickup_weekday,tpep_dropoff_datetime,dropoff_year,dropoff_month,dropoff_day,dropoff_weekday
0,0,2016-03-01 00:53:35,2016,3,1,1,2016-03-01 00:57:11,2016,3,1,1
1,1,2016-03-01 00:15:13,2016,3,1,1,2016-03-01 00:15:26,2016,3,1,1
2,2,2016-03-01 00:15:46,2016,3,1,1,2016-03-01 00:16:00,2016,3,1,1
3,3,2016-03-01 00:16:17,2016,3,1,1,2016-03-01 00:16:23,2016,3,1,1
4,4,2016-03-01 00:16:45,2016,3,1,1,2016-03-01 00:18:14,2016,3,1,1


In [36]:
dim_rate_code = df[['RatecodeID']].drop_duplicates().reset_index(drop=True)

rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

dim_rate_code = dim_rate_code.assign(
    rate_code_id = dim_rate_code.index,\
    rate_code_name = dim_rate_code['RatecodeID'].map(rate_code_type)
)

dim_rate_code = dim_rate_code[['rate_code_id', 'RatecodeID', 'rate_code_name']]

dim_rate_code

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,3,Newark
2,2,2,JFK
3,3,5,Negotiated fare
4,4,4,Nassau or Westchester
5,5,6,Group ride
6,6,99,NaN


In [37]:
dim_payment_type = df[['payment_type']].drop_duplicates().reset_index(drop=True)

payment_types = {
    1 : "Credit Card",
    2 : 'Cash',
    3: 'No charge',
    4: 'Dispute',
    5 : 'Unknown',
    6 : 'Voided trip'
}

dim_payment_type = dim_payment_type.assign(
    payment_type_id = dim_payment_type.index,\
    payment_type_name = dim_payment_type.payment_type.map(payment_types)
    
)

dim_payment_type = dim_payment_type[['payment_type_id', 'payment_type', 'payment_type_name']]

dim_payment_type

,payment_type_id,payment_type,payment_type_name
0,0,3,No charge
1,1,2,Cash
2,2,1,Credit Card
3,3,4,Dispute


In [38]:
df_taxi_zones = pd.read_csv('../data/taxi_zones.csv', usecols=['zone', 'LocationID','borough'])

df_taxi_zones_lookup = df_taxi_zones.drop_duplicates()
df_taxi_zones_lookup = df_taxi_zones_lookup[['LocationID', 'borough','zone']]

dim_dropoff_location = df[['DOLocationID']].drop_duplicates().reset_index(drop=True)

dim_dropoff_location = dim_dropoff_location.assign(
    dropoff_location_id = dim_dropoff_location.index
)

dim_dropoff_location = dim_dropoff_location.merge(df_taxi_zones_lookup, how='left', left_on= 'DOLocationID', right_on='LocationID')

dim_dropoff_location = dim_dropoff_location.rename(columns={'zone' : 'dropoff_location_zone',
                                     'borough' : 'dropoff_location_borough'})

dim_dropoff_location = dim_dropoff_location[['dropoff_location_id', 'DOLocationID', 'dropoff_location_zone','dropoff_location_borough']]

dim_dropoff_location

,dropoff_location_id,DOLocationID,dropoff_location_zone,dropoff_location_borough
0,0,226,Sunnyside,Queens
1,1,145,Long Island City/Hunters Point,Queens
2,2,264,NaN,NaN
3,3,151,Manhattan Valley,Manhattan
4,4,74,East Harlem North,Manhattan
...,...,...,...,...
257,257,5,Arden Heights,Staten Island
258,258,2,Jamaica Bay,Queens
259,259,105,NaN,NaN
260,260,99,Freshkills Park,Staten Island


In [ ]:
df_taxi_zones = pd.read_csv('../data/taxi_zones.csv', usecols=['zone', 'LocationID','borough'])

df_taxi_zones_lookup = df_taxi_zones.drop_duplicates()
df_taxi_zones_lookup = df_taxi_zones_lookup[['LocationID', 'borough','zone']]

pickup_location_dim = df[['PULocationID']].drop_duplicates().reset_index(drop=True)

pickup_location_dim = pickup_location_dim.assign(
    pickup_location_id = pickup_location_dim.index
)

pickup_location_dim = pickup_location_dim.merge(df_taxi_zones_lookup, how='left', left_on= 'PULocationID', right_on='LocationID')

pickup_location_dim = pickup_location_dim.rename(columns={'zone' : 'pickup_location_zone',
                                     'borough' : 'pickup_location_borough'})

pickup_location_dim = pickup_location_dim[['pickup_location_id', 'PULocationID', 'pickup_location_zone','pickup_location_borough']]

pickup_location_dim

In [31]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [32]:
fare_dim = df[['fare_amount', 'extra', 'mta_tax',
               'tip_amount', 'tolls_amount', 'improvement_surcharge',
               'total_amount', 'congestion_surcharge', 'airport_fee']]\
                .drop_duplicates().reset_index(drop=True)

fare_dim = fare_dim.assign(
    fare_id = fare_dim.index
)

fare_dim = fare_dim[['fare_id','fare_amount', 'extra', 'mta_tax',
                     'tip_amount', 'tolls_amount', 'improvement_surcharge',
                     'total_amount', 'congestion_surcharge', 'airport_fee']]
fare_dim

,fare_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,0,4.0,0.5,0.5,0.00,0.00,0.3,5.30,None,None
1,1,2.5,0.5,0.5,0.00,0.00,0.3,3.80,None,None
2,2,3.0,0.5,0.5,0.00,0.00,0.3,4.30,None,None
3,3,6.5,0.5,0.5,0.00,0.00,0.3,7.80,None,None
4,4,8.5,0.5,0.5,1.96,0.00,0.3,11.76,None,None
...,...,...,...,...,...,...,...,...,...,...
83183,83183,25.0,0.5,0.5,11.00,0.00,0.3,37.30,None,None
83184,83184,50.0,0.0,0.0,1.00,10.54,0.3,61.84,None,None
83185,83185,34.0,0.5,0.5,4.44,5.54,0.3,45.28,None,None
83186,83186,76.0,0.5,0.5,42.70,0.00,0.3,120.00,None,None


In [40]:
#dim store and forward
store_and_forward_dim = df[['store_and_fwd_flag']].drop_duplicates().reset_index(drop=True)

flag_bool_type = {
    'N': 0,
    'Y': 1,
}

store_and_forward_dim = store_and_forward_dim.assign(
    store_and_forward_id = store_and_forward_dim.index,\
    store_and_forward_bool = store_and_forward_dim['store_and_fwd_flag'].map(flag_bool_type)
    )

store_and_forward_dim.store_and_forward_bool = store_and_forward_dim.store_and_forward_bool.astype('boolean')

store_and_forward_dim = store_and_forward_dim[['store_and_forward_id', 'store_and_forward_bool', 'store_and_fwd_flag']]

store_and_forward_dim

,store_and_forward_id,store_and_forward_bool,store_and_fwd_flag
0,0,False,N
1,1,True,Y


In [35]:
df.store_and_fwd_flag.value_counts()

store_and_fwd_flag
N    12137085
Y       66739
Name: count, dtype: int64

In [ ]:
fct_table = df\
    .merge(vendor_dim[['vendor_id', 'VendorID']], how='left', on='VendorID')\
    .merge(datetime_dim[['datetime_id','tpep_pickup_datetime','tpep_dropoff_datetime']], how='left', on=['tpep_pickup_datetime','tpep_dropoff_datetime'])\
    .merge(rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']], how='left', on=['RatecodeID'])\
    .merge(payment_type_dim[['payment_type_id','payment_type','payment_type_name']], how='left', on=['payment_type'])\
    .merge(dropoff_location_dim[['dropoff_location_id','DOLocationID','dropoff_location_zone', 'dropoff_location_borough']], how='left', on=['DOLocationID'])\
    .merge(pickup_location_dim[['pickup_location_id','PULocationID','pickup_location_zone', 'pickup_location_borough']], how='left', on=['PULocationID'])\
    .merge(trip_distance_dim[['trip_distance_id','trip_distance']], how='left', on=['trip_distance'])\
    .merge(passenger_count_dim[['passenger_count_id','passenger_count']], how='left', on=['passenger_count'])\
    .merge(store_and_forward_dim[['store and fwd id','store_and_fwd_flag']], how='left', on=['store_and_fwd_flag'])\
    .merge(fare_dim, how='left', on=['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge','total_amount', 'congestion_surcharge', 'airport_fee'])\
    .assign(event_id = lambda x: x.index)
        
columns_in_fct_table = ['event_id','vendor_id', 'datetime_id','passenger_count_id','trip_distance_id','pickup_location_id',
                        'dropoff_location_id','rate_code_id','payment_type_id','store_and_fwd_id', 'fare_id']
    
fct_table = fct_table.loc[:, columns_in_fct_table]

fct_table